In [28]:
import pandas as pd
import numpy as np

In [29]:
pd.read_csv('spotify_millsongdata.csv')

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...


In [30]:
df = pd.read_csv('spotify_millsongdata.csv')
df.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [31]:
df.shape

(57650, 4)

In [32]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [33]:
df.duplicated().sum()

0

In [34]:
df = df[['artist', 'song', 'text']]
df.head()

,artist,song,text
0,ABBA,Ahe's My Kind Of Girl,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante","Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,Making somebody happy is a question of give an...


In [35]:
df['text'][0]

"Look at her face, it's a wonderful face  \r\nAnd it means something special to me  \r\nLook at the way that she smiles when she sees me  \r\nHow lucky can one fellow be?  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?  \r\n  \r\nAnd when we go for a walk in the park  \r\nAnd she holds me and squeezes my hand  \r\nWe'll go on walking for hours and talking  \r\nAbout all the things that we plan  \r\n  \r\nShe's just my kind of girl, she makes me feel fine  \r\nWho could ever believe that she could be mine?  \r\nShe's just my kind of girl, without her I'm blue  \r\nAnd if she ever leaves me what could I do, what could I do?\r\n\r\n"

In [36]:
df['text'] = df['text'].str.lower().str.replace('\r\n', ' ')

In [37]:
df = df.head(50000)

In [38]:
df.shape

(50000, 3)

In [39]:
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize

In [40]:
stemmer = PorterStemmer()

In [41]:
def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [42]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [43]:
df['text'].head()

0    look at her face , it 's a wonder face and it ...
1    take it easi with me , pleas touch me gentli l...
2    i 'll never know whi i had to go whi i had to ...
3    make somebodi happi is a question of give and ...
4    make somebodi happi is a question of give and ...
Name: text, dtype: object

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [50]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
vector = tfidvector.fit_transform(df['text'])

In [51]:
vector

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2740707 stored elements and shape (50000, 57114)>

In [52]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

batch_size = 10000
num_batches = int(np.ceil(vector.shape[0] / batch_size))  
similarity_results = []

for i in range(num_batches):
    start_index = i * batch_size
    end_index = min((i + 1) * batch_size, vector.shape[0])

    current_batch = vector[start_index:end_index]

    similarity_batch = cosine_similarity(current_batch)

    similarity_results.append(similarity_batch)



In [53]:
similarity_results = np.concatenate(similarity_results, axis=0)

In [54]:
similarity_results.shape

(50000, 10000)

In [55]:
similarity_results

array([[1.        , 0.02840853, 0.01363084, ..., 0.08140276, 0.02615301,
        0.06623395],
       [0.02840853, 1.        , 0.0063234 , ..., 0.01148545, 0.00937928,
        0.05692629],
       [0.01363084, 0.0063234 , 1.        , ..., 0.07349687, 0.01043806,
        0.07785117],
       ...,
       [0.02915728, 0.01472295, 0.        , ..., 1.        , 0.01448285,
        0.00390496],
       [0.02321455, 0.02175443, 0.        , ..., 0.01448285, 1.        ,
        0.00517645],
       [0.01228549, 0.04101825, 0.02977676, ..., 0.00390496, 0.00517645,
        1.        ]])

In [70]:
def recommendation(song):
    idx = df[df['song'] == song].index[0]
    distances = sorted(list(enumerate(similarity_results[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:6]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [71]:
recommendation('Bang')

['Sea Of Dreams',
 'The Prime Of Your Love',
 'Bang-A-Boomerang',
 'Endless Love',
 "Janie's Got A Gun"]

In [57]:
import pickle
pickle.dump(similarity_results, open('similarity.pkl', 'wb'))
print('similarity saved as pkl file')
pickle.dump(df, open('song_dict.pkl', 'wb'))
print('song_dict saved as pkl file')

similarity saved as pkl file
song_dict saved as pkl file
